# Tensorflow basic
----------------------------
[Getting Started With TensorFlow](https://www.tensorflow.org/get_started/get_started)  
You might think of TensorFlow Core programs as consisting of two discrete sections:
1. Building the computational graph.
2. Running the computational graph.
A **computational graph** is a series of TensorFlow operations arranged into a graph of nodes.




In [ ]:
import tensorflow as tf
import os

In [ ]:
# tf.graph visualization util function
# code from 'https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter'
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
# Create two constants.
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly

print(node1, node2)

Notice that printing the **nodes does not output the values 3.0 and 4.0** as you might expect. Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. To actually evaluate the nodes, we must run **the computational graph within a session.** A session encapsulates the control and state of the TensorFlow runtime.

In [ ]:
sess = tf.Session()
ret1, ret2 = sess.run([node1, node2])
print(ret1, ret2)

We can build more complicated computations by combining **Tensor** nodes with operations (Operations are also nodes.). For example, we can add our two constant nodes and produce a new graph as follows:



In [ ]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ", sess.run(node3))

# Let's visualize these nodes
show_graph(tf.get_default_graph())

As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as **placeholders**. A **placeholder** is a promise to provide a value later.



In [ ]:
graph2 = tf.Graph()
with graph2.as_default():
    a = tf.placeholder(tf.float32)
    b = tf.placeholder(tf.float32)
    adder_node = a + b  # + provides a shortcut for tf.add(a, b)
    
print(a)
print(b)
print(adder_node)

In [ ]:
print(graph2)
print(tf.get_default_graph()) # default_graph
print(a.graph) # graph2

In [ ]:
# let's feed some values to graph
sess2 = tf.Session(graph=graph2)
print(sess2.run(adder_node, feed_dict={a: 3, b:4.5}))
print(sess2.run(adder_node, feed_dict={a: [1,3], b: [2, 4]}))

# Let's visualize these nodes
show_graph(graph2)

In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. **Variables** allow us to add **trainable parameters** to a graph. They are constructed with a type and initial value:

In [ ]:
sess.close(), sess2.close() # close session
tf.reset_default_graph() # reset our default graph
sess = tf.Session() # We need to renew Session because the first graph is not valid

In [ ]:
W = tf.Variable([.3], tf.float32, name='weight')
b = tf.Variable([-.3], tf.float32, name='bias')
x = tf.placeholder(tf.float32, name='input')
linear_model = W * x + b

print(W)
print(b)
print(x)
print(linear_model)

Variables are **not initialized** when you call **tf.Variable**. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [ ]:
# you can use tf.global_variables_initializer().run() instead
# when using tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init) 

In [ ]:
print(sess.run(linear_model, {x:[1,2,3,4]}))
show_graph(tf.get_default_graph())

In [ ]:
print(sess.run([W, b]))

# Simple linear regression
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

## Tensorflow.Saver
------------------------
[Tensorflow saver tutorial](https://www.tensorflow.org/programmers_guide/variables)   
[Hello, TensorFlow!](https://www.oreilly.com/learning/hello-tensorflow)

We can use **tf.Saver** to save our variables.

In [ ]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
show_graph(tf.get_default_graph())

In [ ]:
PATH = 'ckpt_linear'
MODEL_PATH = os.path.join(PATH, 'linear')
if not os.path.exists(PATH):
    os.makedirs(PATH)
saver.save(sess, MODEL_PATH)

In [ ]:
sess.run(init)
print(W.eval(sess), b.eval(sess)) # equivalent with sess.run([W, b])

In [ ]:
saver.restore(sess, MODEL_PATH)
print(W.eval(sess), b.eval(sess))

Now, we can save our optimized variables. But, it does **not reconstruct our models.**

In [ ]:
# reset graph
sess.close()
tf.reset_default_graph()
sess = tf.Session()

W = tf.Variable([.3], tf.float32, name='weight')
b = tf.Variable([-.3], tf.float32, name='bias')

saver = tf.train.Saver()
saver.restore(sess, MODEL_PATH)
show_graph(tf.get_default_graph()) # This model has only two variables!

So we need to restore model from meta data of the latest saved graph. The method, tf.Saver.save(), automatically created meta data of the model with **'(my_model_name).meta'**. So, we can restore our model as following codes:

In [ ]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

saver = tf.train.import_meta_graph(MODEL_PATH + '.meta')
saver.restore(sess, tf.train.latest_checkpoint(PATH))

# get the variable by name
print(sess.run(['weight:0', 'bias:0']))

# restore python object to feed new data
graph = tf.get_default_graph()
x = graph.get_tensor_by_name('input:0')
linear_model = graph.get_tensor_by_name('add:0')

print(sess.run(linear_model, feed_dict={x: [1, 2, 3, 4]}))
show_graph(graph)